In [1]:
import library_py2 as lib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# from sklearn.metrics import roc
# from sklearn.model_selection import traintestsplit
# from sklearn.preprocessing import 
import json
import warnings
# from nltk.punkt impor
import nltk
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')
import string
from nltk.stem.wordnet import WordNetLemmatizer, wordnet
from nltk.stem.porter import PorterStemmer
from nltk.collocations import *
from nltk import FreqDist
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
import re
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
warnings.filterwarnings(action='once')
%matplotlib inline
pd.set_option('display.max_columns', 999)
%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence
from keras.utils import to_categorical

In [4]:
df = pd.read_pickle('lstm_df.pkl')

In [45]:
df.head()

,context_lstm,question_lstm,answers_lstm,text
0,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,when do beyonce start become popular,in the late 1990s,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
1,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,what area do beyonce compete in when she be gr...,singing and dancing,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
2,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,when do beyonce leave destinys child and becom...,2003,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
3,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,in what city and state do beyonce grow up,houston texas,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
4,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,in which decade do beyonce become famous,late 1990s,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...


In [6]:
df.drop(columns=['plausible_answers_lstm'], inplace=True)

In [15]:
vocabulary = lib.get_total_vocab(df, columns=['context_lstm', 'question_lstm'])

In [16]:
len(vocabulary)

92265

In [30]:
vocabulary.keys()

dict_keys(['', '70–75', 'iv', 'ghindr', 'topsoil', 'christiansen', 'maiores', 'panelization', 'expansion', 'dedicatory', 'neustria', 'rails', '龍', 'aerialism', 'xray', 'uns', 'measurable', 'expublic', 'disasterridden', 'apartmentretail', '6041', 'baroque', 'śūnyatā', 'mercer', 'enclaves', 'oetats', 'indestructible', 'valse', 'moha', 'cargocarriers', '765', 'selfdeclared', 'rosecrans', 'council', 'cascading', 'thinnest', 'enoch', 'streetfronts', 'psip', 'coulmiers', 'reconciling', '210000', 'documentoriented', 'mozambican', 'amgen', 'legs', 'hoofdklasse', 'banquet', '770', '22716', '1942–43', 'shaoxing', 'gaslasers', 'folkore', '“essential', 'labelle', 'sabio', 'fajr5', 'šlėkta', '19acre', 'stc', 'slumber', 'liangs', 'carys', 'nonconfrontational', 'midnapore', 'tubenose', 'malaysianborn', 'somersaulting', 'sommerzeit', 'sasanin', 'sklabēnoi', 'fitzgerald', 'making', 'libraries', 'khmelnytsky', 'ʰp', 'diadochi', 'covalency', 'gustafson', 'only', 'telefónica', 'officers”', 'neumaan', 'der

In [17]:
df.context_lstm.nunique()

18875

In [38]:
def file_to_word_ids(vocabulary, data, columns):
    txt = []
    for col in columns:
        for i in data[col]:
            j = i.split(' ')
            for k in j:
                txt.append(k)
    word2id_dict = vocabulary
    return [word2id_dict[word] for word in vocabulary if word in txt]


In [20]:
wod = file_to_word_ids(vocabulary, df[:10], columns=['context_lstm', 'question_lstm'])

In [21]:
wod

[0,
 243,
 77160,
 2470,
 62271,
 3276,
 3290,
 781,
 7710,
 8070,
 9709,
 9768,
 63389,
 10070,
 11602,
 12164,
 12233,
 12577,
 12702,
 14935,
 15666,
 1450,
 18318,
 19085,
 3192,
 19507,
 20202,
 23530,
 24331,
 24494,
 26378,
 26496,
 28315,
 28578,
 30449,
 31454,
 31522,
 31827,
 33849,
 34368,
 34909,
 36575,
 37456,
 38025,
 38079,
 38326,
 39302,
 40091,
 40363,
 40924,
 42856,
 38470,
 23138,
 46913,
 47737,
 47784,
 23752,
 49569,
 24097,
 50698,
 51284,
 51650,
 70433,
 54099,
 54687,
 55661,
 71069,
 56794,
 58519,
 58803,
 59066,
 60589,
 62540,
 57137,
 64827,
 11027,
 65417,
 65860,
 66175,
 66931,
 67252,
 67406,
 67844,
 68491,
 69905,
 71672,
 71830,
 71881,
 72896,
 74546,
 76236,
 78483,
 74737,
 81410,
 81568,
 82972,
 84042,
 84094,
 31557,
 87699,
 87766,
 88916,
 89221,
 90150,
 34853,
 90668,
 91239]

In [25]:
df['text'] = df['context_lstm'] + ' ' + df['question_lstm'] 

In [41]:
def load_data():
    # get the data paths
    train_df = df.iloc[:round(0.6*len(df))]# will automate this step at a further pt
    valid_df = df.iloc[round(0.6*len(df)):round(0.8*len(df))]
    test_df = df.iloc[round(0.8*len(df)):]

    # build the complete vocabulary, then convert text data to list of integers
    vocabulary = lib.get_total_vocab(train_df, columns=['text'])
    train_data = lib.file_to_word_ids(vocabulary, train_df, columns=['text'])
    valid_data = lib.file_to_word_ids(vocabulary, valid_df, columns=['text'])
    test_data = lib.file_to_word_ids(vocabulary, test_df, columns=['text'])
    vocabulary_size = len(vocabulary)
    reversed_dictionary = dict(zip(vocabulary.values(), vocabulary.keys()))

    print(train_data[:5])
    print(word_to_id)
    print(vocabulary_size)
    print(" ".join([reversed_dictionary[x] for x in train_data[:10]]))
    return train_data, valid_data, test_data, vocabulary, reversed_dictionary



In [ ]:
train_data, valid_data, test_data, vocabulary, reversed_dictionary = load_data()

In [ ]:
class KerasBatchGenerator(object):
    def __init__(self, data, num_steps, batch_size, vocabulary, skip_step=5):
        self.data = data
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.vocabulary = vocabulary
        self.current_idx = 0
        self.skip_step = skip_step
        
        
    #     num_steps = nr of words that will be fed into the input layer; i.e. the nr of words the model will use to predict the next word
    #     skip_steps = how far to move the window after the prediction is made

    def generate(self):
        x = np.zeros((self.batch_size, self.num_steps)) # input nn nodes, same dimensions as batchsize x nr of words in one "window"
        y = np.zeros((self.batch_size, self.num_steps, self.vocabulary)) # output nodes
        
        while True:
            for i in range(self.batch_size):
                if self.current_idx + self.num_steps >= len(self.data): # once the ticker goes over the length of data, reset it
                    self.current_idx = 0
                x[i,:] = self.data[self.current_idx:self.current_idx + self.num_steps]
                # input is the segment of data from current index to current index + step-increment
                temp_y = self.data[(self.current_idx + 1) : (self.num_steps + 1) ]
                # converts all the output y into a 1-hot representation
                y[i, :, :] = to_categorical(temp_y, num_classes=self.vocabulary)
                self.current_idx += self.skip_step
                
            yield x, y
            
        # x dimensions: nr of samples we specify in the batch first, number of w to base predictions on 
        # y has 3 dims: batchsize, nr time steps and also size of vocabulary
        
        
train_data_generator = KerasBatchGenerator(train_data, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)
valid_data_generator = KerasBatchGenerator(valid_data, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)

model = Sequential()
model.add(Embedding(vocabulary, hidden_size, input_length=num_steps))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(LSTM(hidden_size, return_sequences=True))
if use_dropout:
    model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(vocabulary)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

model.fit_generator(train_data_generator.generate(), len(train_data)//(batch_size*num_steps), num_epochs,
                        validation_data=valid_data_generator.generate(),
                        validation_steps=len(valid_data)//(batch_size*num_steps))